In [1]:
# Put the correct credentials here and mount S3 bucket
ACCESS_KEY = ""
SECRET_KEY = ""
AWS_BUCKET_NAME = ""
MOUNT_NAME = ""
ENCODED_SECRET_KEY = SECRET_KEY.replace("/", "%2F")

try: 
  dbutils.fs.mount("s3a://%s:%s@%s" % (ACCESS_KEY, ENCODED_SECRET_KEY, AWS_BUCKET_NAME), "/mnt/%s" % MOUNT_NAME)
except:
  pass

In [2]:
# Similar to SparkContext, for SparkSQL you need a SparkSession
from pyspark.sql import SparkSession
# Also all the functions (select, where, groupby) needs to be imported
from pyspark.sql.functions import *

In [3]:
# Get spark session
spark = SparkSession.builder.getOrCreate()

In [4]:
# read data into dataframe
flights_df = spark.read.csv("/mnt/%s/flights.csv" % MOUNT_NAME, header=True)

### DataFrames Operations

In this part you will learn how to programmatically use the SQL capabilities of DataFrame. For the full list of documentation: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql

In [6]:
# You can use the select method to grab specific columns
display(flights_df.select(['AIRLINE','DEPARTURE_DELAY']))

In [7]:
# You can change the Data type of any column by casting them to your desired data type
# First you have to import that data type from pyspark.sql.types
from pyspark.sql.types import IntegerType
# Then you can use withColumn() to apply / cast each row of the column (Notice how the square bracket annotation is used)
flights_df = flights_df.withColumn("DEPARTURE_DELAY", flights_df['DEPARTURE_DELAY'].cast(IntegerType()))
# take a look at the schema now
flights_df.select(['AIRLINE','DEPARTURE_DELAY']).printSchema()

In [8]:
# You can use the filter() here to filter on a condition (just like we did with RDD!)
# As you can see there is a lot of null values of departure. For empty strings '' values couldn't be converted to Integer and hence they were turned to Null values
flights_df.filter(flights_df.DEPARTURE_DELAY.isNull()).count()

In [9]:
# that is around 1.4 % of our total data!
(float(flights_df.filter(flights_df.DEPARTURE_DELAY.isNull()).count()) / flights_df.count()) * 100

In [10]:
# get the mean of departure delay
flights_df.filter(flights_df.DEPARTURE_DELAY.isNotNull()).agg(avg(flights_df.DEPARTURE_DELAY)).collect()

In [11]:
# we can specify that we want to fillna for only one column by using the subset argument in the following way
flights_df = flights_df.fillna(9.37, subset=['DEPARTURE_DELAY'])

In [12]:
# now if you take a look at the count of null values you can be convinced our previous steps worked!
flights_df.filter(flights_df.DEPARTURE_DELAY.isNull()).count()

### User Defined Functions (UDF)
Similar to custom functions for Map, you can write user defined function to transform one or more columns. 
More about UDF on https://docs.databricks.com/spark/latest/spark-sql/udf-in-python.html

In [14]:
# Using UDF is a three step process. Before anything you will need to import the udf library
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

In [15]:
# If you can express your user defined function as lambda then you can register the UDF and define it in one line like below
format_delay_udf = udf(lambda delay: delay if delay > 0 else 0, IntegerType())

In [16]:
# Otherwise you can first write your function
def format_delay(delay):
  return delay if delay > 0 else 0
# and then register it as an UDF with the return type declared
format_delay_udf = udf(format_delay, IntegerType())

In [17]:
# Now you can use withColumn to apply the UDF over every row for DEPARTURE_DELAY column
flights_df = flights_df.withColumn('DEPARTURE_DELAY', format_delay_udf(flights_df['DEPARTURE_DELAY']))

### Group By
The GROUP BY statement is used with **aggregate functions (COUNT, MAX, MIN, SUM, AVG)** to group the result-set by one or more columns.

In [19]:
# For instance, we can group by the DEPARTURE DELAY over AIRLINE and aggregate over the average value
display(flights_df.groupBy('AIRLINE').agg(avg('DEPARTURE_DELAY').alias('AVG_DEPARTURE_DELAY')))

In [20]:
# we can also display how many times each airline flew in 2015 by using the count aggregate function
display(flights_df.groupBy('AIRLINE').agg(count('AIRLINE').alias('FLIGHT_COUNT')))

In [21]:
# We can also Group By over different departure airports, lets limit to top 20!
dep_delay_by_airport_df = flights_df.groupBy('ORIGIN_AIRPORT').agg(avg('DEPARTURE_DELAY').alias('AVG_DEPARTURE_DELAY'))
display(dep_delay_by_airport_df.sort('AVG_DEPARTURE_DELAY', ascending=False).limit(10))

### Joins
A JOIN clause is used to combine rows from two or more tables, based on a related column between them. Here are the a few basic types of joins explained:

* (INNER) JOIN: Returns records that have matching values in both tables
* LEFT (OUTER) JOIN: Return all records from the left table, and the matched records from the right table
* RIGHT (OUTER) JOIN: Return all records from the right table, and the matched records from the left table
* FULL (OUTER) JOIN: Return all records when there is a match in either left or right table

Spark Supports more than just basic joins however. With the latest spark you get: inner, cross, outer, full, full_outer, left, left_outer, right, right_outer, left_semi, and left_anti joins! Take a look in  https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#join for more details.

In [23]:
# lets use the airports csv file to make sense of the airports in our previous results
airports_df = spark.read.csv("/mnt/%s/airports.csv" % MOUNT_NAME, header=True)
display(airports_df)

In [24]:
# we do an inner join to get more information about each airport
dep_delay_by_airport_df = dep_delay_by_airport_df.join(airports_df, dep_delay_by_airport_df['ORIGIN_AIRPORT'] == airports_df['IATA_CODE'])

In [25]:
# you can use sort() and then limit() to get the top n number of results back
display(dep_delay_by_airport_df.sort('AVG_DEPARTURE_DELAY', ascending=False).limit(10))

In [26]:
# unfortunately we have a few extra entries for state in our dataset
dep_delay_by_airport_df.select('STATE').distinct().count()

### Visualization in DataBricks
Here we take advantage of the built in plots in Databrick's notebook to visualize average delay per state. As we have some state entry in our list, I will load a csv of States in USA and do an inner join to get rid of invalid entries.

In [28]:
# load the usa states dataset
state_codes_df = spark.read.csv("/mnt/%s/usa_state_codes.csv" % MOUNT_NAME, header=True)

In [29]:
# we use the display and then plot option to visualize the average delay per state
display(dep_delay_by_airport_df.join(state_codes_df, dep_delay_by_airport_df['STATE']==state_codes_df['Code'],how='inner'))

In [30]:
x= dep_delay_by_airport_df.join(state_codes_df, dep_delay_by_airport_df['STATE']==state_codes_df['Code'],how='inner')

### Challenge: Can you make the same geo-plot for arrival delay?